In [1]:
def resample_to_constant_volume_flex(data):
    def volumeSum_threshold():
        cumsum = 0
        val = yield cumsum
        while True:
            if (cumsum < val[1] ) or ("15:29" in val[2]) and ("09:15" not in val[2]):
                cumsum += val[0]
                if ("15:29" in val[2]):
                    cumsum += val[1]
                val = yield cumsum
            else:
                cumsum = val[0]
                val = yield cumsum
    a = volumeSum_threshold()
    next(a)
    data["volume_per_candle"] = data["volume"].rolling(window=20).mean().rolling(window = 110).quantile(0.95)
    data["volume_per_candle"].fillna(method="bfill",inplace=True)
    data["cumsum"] = data["volume"].cumsum()

    data["volume_vol"] = [a.send(v) for v in np.array(data[["volume","volume_per_candle","date"]])]
    
    data["thesholdBreak"] = data['volume_vol']>data["volume_per_candle"]
    
    data["stamp"] = data['date'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").timestamp())
    
    data["close_vol"] = data["close"]*data["thesholdBreak"].astype(int)
    data["end"] = data["stamp"]*data["thesholdBreak"].astype(int)
    
    data["open_vol"] = data["open"]*((data["thesholdBreak"]).astype(int).shift(1))
    data["open_vol"].iloc[0] = data["open"].iloc[0]
    data["open_vol"] = data['open_vol'].where(data["open_vol"] != 0, np.nan)
    data["open_vol"].fillna(method="ffill",inplace=True)
    
    
    data["start"] = data["stamp"]*((data["thesholdBreak"]).astype(int).shift(1))
    data["start"].iloc[0] = data["stamp"].iloc[0]
    data["start"] = data['start'].where(data["start"] != 0, np.nan)
    data["start"].fillna(method="ffill",inplace=True)
    
    data["close_vol"] = data['close_vol'].where(data["close_vol"] != 0, np.nan)
    data["close_vol"].iloc[-1] = data['close'].iloc[-1]
    data["close_vol"].fillna(method="bfill",inplace=True)
    
    data["end"] = data['end'].where(data["end"] != 0, np.nan)
    data["end"].iloc[-1] = data['end'].iloc[-1]
    data["end"].fillna(method="bfill",inplace=True)
    
    def high_():
        hh = 0
        val = yield hh
        while True:
            if val[0]:
                hh = max(hh,val[1])
                val = yield hh
                hh = 0
            else:
                hh = max(hh,val[1])
                val = yield hh
    h = high_()
    next(h)
    def low_():
        ll = np.inf
        val = yield ll
        while True:
            if val[0]:
                ll = min(ll,val[1])
                val = yield ll
                ll = np.inf
            else:
                ll = min(ll,val[1])
                val = yield ll
    l = low_()
    next(l)
    
    data["high_vol"]=[h.send(v) for v in np.array(data[["thesholdBreak","high"]])]
    data["low_vol"] =[l.send(v) for v in np.array(data[["thesholdBreak","low"]])]
    data["open"] = data["open_vol"]
    data["high"] = data["high_vol"]
    data["low"] = data["low_vol"]
    data["close"] = data["close_vol"]
    data["volume"] = data["volume_vol"]
    data["timeDiff"] = data["end"] - data["start"]
    
    
    return data[["date","open","high","low","close","volume_per_candle","volume","timeDiff","start","end"]][data["thesholdBreak"]]

In [24]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objs as go


In [25]:
imb_candle_sam_new = ['2016-11-01 14:23:00', '2016-11-02 09:15:00', '2016-11-02 15:10:00', '2016-11-03 13:12:00', '2016-11-03 13:17:00', '2016-11-03 15:15:00', '2016-11-10 09:36:00', '2016-11-10 09:39:00', '2016-11-10 14:50:00', '2016-11-11 12:30:00', '2016-11-17 09:31:00', '2016-11-17 13:45:00', '2016-11-17 13:50:00', '2016-11-18 11:35:00', '2016-11-18 15:28:00', '2016-11-25 12:30:00', '2016-11-28 13:36:00', '2016-12-01 14:21:00', '2016-12-02 09:15:00', '2016-12-02 13:09:00', '2016-12-19 14:37:00', '2016-12-20 13:00:00', '2016-12-21 15:01:00', '2016-12-21 15:12:00', '2016-12-21 15:14:00', '2016-12-28 13:18:00', '2016-12-28 14:33:00', '2016-12-29 15:03:00', '2017-01-03 09:34:00', '2017-01-09 12:39:00', '2017-01-09 12:59:00', '2017-01-09 13:19:00', '2017-01-09 14:51:00', '2017-01-13 13:56:00', '2017-01-16 15:04:00', '2017-01-18 14:26:00', '2017-01-18 15:22:00', '2017-01-25 14:36:00', '2017-01-27 12:42:00', '2017-02-08 13:44:00', '2017-02-08 14:33:00', '2017-02-10 15:20:00', '2017-02-20 09:16:00', '2017-02-23 15:03:00', '2017-03-03 12:00:00', '2017-03-14 14:56:00', '2017-03-14 15:26:00', '2017-03-24 13:26:00', '2017-03-27 14:55:00', '2017-03-27 14:59:00', '2017-03-29 14:18:00', '2017-04-05 09:19:00', '2017-04-10 12:43:00', '2017-04-17 14:14:00', '2017-04-18 13:22:00', '2017-04-26 14:26:00', '2017-04-26 14:31:00', '2017-05-09 15:22:00', '2017-05-10 10:33:00', '2017-05-10 15:27:00', '2017-05-11 15:01:00', '2017-05-12 14:51:00', '2017-05-16 09:17:00', '2017-05-16 09:26:00', '2017-05-18 09:36:00', '2017-05-18 14:37:00', '2017-05-22 14:14:00', '2017-05-24 13:53:00', '2017-05-31 15:01:00', '2017-06-02 13:08:00', '2017-06-08 09:57:00', '2017-06-15 13:14:00', '2017-06-22 15:01:00', '2017-07-04 10:12:00', '2017-07-07 14:32:00', '2017-07-21 12:26:00', '2017-07-24 12:52:00', '2017-09-14 12:57:00', '2017-09-14 13:00:00', '2017-09-15 13:37:00', '2017-09-20 09:18:00', '2017-09-20 12:08:00', '2017-09-20 14:58:00', '2017-09-22 09:20:00', '2017-09-25 09:58:00', '2017-09-26 09:15:00', '2017-09-26 09:27:00', '2017-09-27 14:06:00', '2017-09-28 09:25:00', '2017-09-28 15:11:00', '2017-09-29 15:03:00', '2017-10-03 13:26:00', '2017-10-09 15:05:00', '2017-10-09 15:09:00', '2017-10-09 15:23:00', '2017-10-11 12:21:00', '2017-10-11 13:30:00', '2017-10-11 15:10:00', '2017-10-16 11:00:00', '2017-10-18 10:59:00', '2017-10-26 09:25:00', '2017-10-27 11:36:00', '2017-11-06 09:45:00', '2017-11-06 15:19:00', '2017-11-07 13:45:00', '2017-11-13 14:33:00', '2017-11-14 11:32:00', '2017-11-17 15:22:00', '2017-11-21 13:41:00', '2017-11-23 13:14:00', '2017-11-24 09:17:00', '2017-11-29 14:29:00', '2017-11-29 15:23:00', '2017-12-01 14:54:00', '2017-12-01 15:00:00', '2017-12-04 09:23:00', '2017-12-06 14:21:00', '2017-12-12 09:16:00', '2017-12-12 14:34:00', '2017-12-13 14:46:00', '2017-12-14 12:26:00', '2017-12-20 09:15:00', '2017-12-28 15:05:00', '2017-12-28 15:12:00', '2017-12-29 14:19:00', '2018-01-01 14:56:00', '2018-01-01 15:00:00', '2018-01-01 15:03:00', '2018-01-05 14:31:00', '2018-01-30 15:00:00', '2018-02-01 12:41:00', '2018-02-16 10:48:00', '2018-02-19 12:41:00', '2018-02-19 13:34:00', '2018-02-20 14:58:00', '2018-02-22 09:26:00', '2018-02-22 09:31:00', '2018-02-28 15:18:00', '2018-03-06 09:16:00', '2018-03-19 15:04:00', '2018-04-10 15:01:00', '2018-04-10 15:10:00', '2018-04-11 15:29:00', '2018-04-19 13:32:00', '2018-04-19 15:00:00', '2018-04-20 09:15:00', '2018-05-08 13:01:00', '2018-05-21 09:15:00', '2018-05-22 09:15:00', '2018-05-22 13:58:00', '2018-05-23 09:16:00', '2018-05-24 09:15:00', '2018-05-29 11:45:00', '2018-06-04 11:29:00', '2018-06-14 09:15:00', '2018-06-19 15:00:00', '2018-06-19 15:03:00', '2018-06-21 13:49:00', '2018-06-21 14:16:00', '2018-06-26 11:27:00', '2018-07-06 15:04:00', '2018-07-12 12:46:00', '2018-07-12 13:13:00', '2018-07-16 15:00:00', '2018-07-16 15:03:00', '2018-07-16 15:23:00', '2018-07-24 11:37:00', '2018-07-25 09:15:00', '2018-07-25 15:16:00', '2018-07-27 15:05:00', '2018-08-01 12:55:00', '2018-08-02 14:06:00', '2018-08-02 14:14:00', '2018-08-06 11:46:00', '2018-08-08 14:18:00', '2018-08-09 15:20:00', '2018-08-10 13:25:00', '2018-08-13 09:15:00', '2018-08-14 14:24:00', '2018-08-14 14:26:00', '2018-08-14 14:38:00', '2018-08-16 11:43:00', '2018-08-16 15:08:00', '2018-08-29 09:35:00', '2018-08-29 14:11:00', '2018-09-03 12:05:00', '2018-09-04 09:48:00', '2018-09-04 12:07:00', '2018-09-04 13:35:00', '2018-09-10 09:26:00', '2018-09-10 09:50:00', '2018-09-10 14:37:00', '2018-09-18 15:00:00', '2018-09-19 14:11:00', '2018-09-19 14:50:00', '2018-09-21 13:06:00', '2018-09-27 14:54:00', '2018-10-03 09:15:00', '2018-10-03 09:30:00', '2018-10-04 12:47:00', '2018-10-09 13:45:00', '2018-10-17 13:26:00', '2018-10-19 13:14:00', '2018-10-19 13:35:00', '2018-10-22 14:05:00', '2018-10-22 14:09:00', '2018-10-29 12:22:00', '2018-10-30 09:16:00', '2018-11-01 14:25:00', '2018-11-22 09:19:00', '2018-12-06 09:15:00', '2018-12-06 13:45:00', '2018-12-06 14:14:00', '2018-12-10 12:09:00', '2018-12-10 12:18:00', '2018-12-10 14:34:00', '2018-12-10 14:38:00', '2018-12-17 14:53:00', '2019-01-03 15:15:00', '2019-01-04 14:15:00', '2019-01-25 12:17:00', '2019-01-25 12:20:00', '2019-01-25 13:54:00', '2019-01-29 09:16:00', '2019-02-11 09:28:00', '2019-02-12 13:06:00', '2019-02-13 15:02:00', '2019-02-14 09:15:00', '2019-02-14 13:46:00', '2019-02-19 14:05:00', '2019-02-22 15:27:00', '2019-03-01 12:34:00', '2019-03-13 13:14:00', '2019-03-13 13:29:00', '2019-03-19 12:19:00', '2019-03-22 14:15:00', '2019-04-03 14:53:00', '2019-04-04 14:43:00', '2019-04-10 09:20:00', '2019-04-18 15:08:00', '2019-04-23 12:25:00', '2019-04-23 12:40:00', '2019-04-25 15:01:00', '2019-05-02 13:28:00', '2019-05-06 10:49:00', '2019-05-07 11:14:00', '2019-05-14 09:16:00', '2019-05-21 13:34:00', '2019-05-22 09:16:00', '2019-05-27 14:08:00', '2019-06-14 15:14:00', '2019-06-17 13:35:00', '2019-06-20 09:15:00', '2019-06-24 13:18:00', '2019-06-26 09:15:00', '2019-06-27 15:00:00', '2019-06-27 15:05:00', '2019-07-01 14:32:00', '2019-07-03 12:02:00', '2019-07-05 13:20:00', '2019-07-08 13:06:00', '2019-07-15 15:12:00', '2019-07-15 15:28:00', '2019-07-23 14:24:00', '2019-07-23 14:26:00', '2019-07-24 13:01:00', '2019-07-31 12:52:00', '2019-08-01 11:47:00', '2019-08-07 15:21:00', '2019-08-13 13:06:00', '2019-08-27 15:01:00', '2019-08-30 09:30:00', '2019-08-30 11:23:00', '2019-09-06 13:58:00', '2019-09-09 12:11:00', '2019-09-17 13:16:00', '2019-09-17 14:20:00', '2019-09-19 15:22:00', '2019-10-01 14:00:00', '2019-10-04 14:31:00', '2019-10-04 15:00:00', '2019-10-07 14:28:00', '2019-10-15 13:48:00', '2019-10-16 14:01:00', '2019-10-16 14:03:00', '2019-10-22 14:40:00', '2019-10-22 15:04:00', '2019-10-29 15:03:00', '2019-11-01 11:30:00', '2019-11-01 14:25:00', '2019-11-06 15:11:00', '2019-11-15 15:01:00', '2019-11-18 09:15:00', '2019-11-19 13:18:00', '2019-11-20 14:45:00', '2019-11-21 10:22:00', '2019-11-21 15:05:00', '2019-11-22 13:41:00', '2019-11-26 13:19:00', '2019-11-26 13:21:00', '2019-11-26 13:24:00', '2019-11-26 13:52:00', '2019-11-26 14:13:00', '2019-11-27 14:44:00', '2019-11-28 13:32:00', '2019-12-03 13:56:00', '2019-12-03 13:59:00', '2019-12-03 14:05:00', '2019-12-06 13:34:00', '2019-12-31 15:00:00', '2020-01-02 12:54:00', '2020-01-03 14:23:00', '2020-01-03 14:25:00', '2020-01-06 09:16:00', '2020-01-06 12:50:00', '2020-01-16 14:20:00', '2020-01-20 14:45:00', '2020-01-21 09:34:00', '2020-01-22 09:16:00', '2020-01-23 12:54:00', '2020-01-24 14:29:00', '2020-01-31 11:39:00', '2020-01-31 15:02:00', '2020-01-31 15:09:00', '2020-02-01 13:29:00', '2020-02-06 13:51:00', '2020-02-07 12:16:00', '2020-02-12 14:29:00', '2020-02-13 12:35:00', '2020-02-14 14:42:00', '2020-02-24 15:03:00', '2020-03-02 15:02:00', '2020-03-03 11:13:00', '2020-03-09 12:49:00', '2020-03-13 09:18:00', '2020-03-17 15:00:00', '2020-03-18 13:59:00', '2020-03-18 14:57:00', '2020-03-19 09:15:00', '2020-03-30 13:48:00', '2020-04-03 09:28:00', '2020-04-03 14:12:00', '2020-04-15 14:09:00', '2020-04-17 12:08:00', '2020-04-21 13:15:00', '2020-04-21 15:21:00', '2020-04-28 09:34:00', '2020-04-29 15:05:00', '2020-04-29 15:10:00', '2020-04-29 15:15:00', '2020-05-04 13:53:00', '2020-05-05 09:25:00', '2020-05-05 10:36:00', '2020-05-05 12:12:00', '2020-05-05 12:19:00', '2020-05-08 14:57:00', '2020-05-12 09:15:00', '2020-05-15 09:21:00', '2020-05-26 15:18:00', '2020-05-27 09:19:00', '2020-06-09 14:04:00', '2020-06-09 14:07:00', '2020-06-10 13:35:00', '2020-06-15 09:16:00', '2020-06-17 14:20:00', '2020-06-17 14:22:00', '2020-06-17 14:24:00', '2020-06-19 12:58:00', '2020-07-03 13:41:00', '2020-07-08 11:57:00', '2020-07-08 12:00:00', '2020-07-08 14:44:00', '2020-07-13 13:48:00', '2020-07-14 15:01:00', '2020-07-15 11:51:00', '2020-07-22 14:08:00', '2020-07-29 14:44:00', '2020-07-30 11:05:00', '2020-07-30 11:11:00', '2020-07-31 15:01:00', '2020-07-31 15:04:00', '2020-08-03 14:05:00', '2020-08-11 15:16:00', '2020-08-14 13:58:00', '2020-08-14 14:15:00', '2020-08-20 09:37:00', '2020-08-24 12:04:00', '2020-08-28 09:22:00', '2020-08-31 15:04:00', '2020-09-02 12:22:00', '2020-09-03 14:15:00', '2020-09-21 13:20:00', '2020-09-24 09:17:00', '2020-09-24 15:04:00', '2020-10-16 09:16:00', '2020-11-03 14:04:00', '2020-11-17 12:08:00', '2020-11-19 14:28:00', '2020-11-24 13:32:00', '2020-11-25 14:00:00', '2020-12-04 09:18:00', '2020-12-11 12:03:00', '2020-12-15 09:15:00', '2020-12-15 15:07:00', '2020-12-17 15:03:00', '2020-12-18 15:08:00', '2020-12-21 14:31:00', '2020-12-24 09:37:00', '2021-01-01 15:08:00', '2021-01-04 13:57:00', '2021-01-07 15:02:00', '2021-01-07 15:06:00', '2021-01-07 15:08:00', '2021-01-07 15:17:00', '2021-01-12 10:30:00', '2021-01-21 14:43:00', '2021-01-29 15:00:00', '2021-01-29 15:02:00', '2021-02-04 13:36:00', '2021-02-04 13:38:00', '2021-02-12 13:56:00', '2021-02-12 14:00:00', '2021-02-15 14:16:00', '2021-02-22 13:54:00', '2021-02-22 15:01:00', '2021-02-26 15:09:00', '2021-02-26 15:21:00', '2021-03-05 09:15:00', '2021-03-19 09:16:00', '2021-03-19 09:22:00', '2021-03-19 09:26:00', '2021-03-19 15:21:00', '2021-03-31 14:29:00', '2021-04-01 14:54:00', 
                      '2021-04-07 13:16:00', '2021-04-07 15:20:00', '2021-04-09 13:42:00', '2021-04-09 15:29:00', '2021-04-13 09:19:00', '2021-04-19 12:21:00', '2021-04-19 12:25:00', '2021-04-30 15:05:00', '2021-04-30 15:11:00', '2021-05-05 14:26:00', '2021-05-07 13:44:00', '2021-05-12 15:06:00', '2021-05-31 13:33:00', '2021-06-01 14:00:00', '2021-06-01 15:04:00', '2021-06-02 13:02:00', '2021-06-15 15:22:00', '2021-06-16 09:15:00', '2021-06-23 14:29:00']

In [26]:
imb_candle_time = imb_candle_sam_new

In [27]:
len(imb_candle_sam_new)

453

In [28]:
df = pd.read_csv('ASIANPAINT_minute_data_with_indicators.csv' ,usecols = ["date","open","high","low","close","volume"]).iloc[150971:554674]
# data = pd.read_csv("data_2017.txt"))

In [29]:
def ss(x):
        return datetime.strptime(x,"%Y-%m-%d %H:%M:%S+05:30").strftime("%Y-%m-%d %H:%M:%S")
df["date"] = df["date"].apply(lambda x: x.split("+")[0])

In [30]:
df_vol = resample_to_constant_volume_flex(df.copy())
df = df_vol
df = df.reset_index()

In [31]:
def is_body_size_greater_than_3x_average(dataframe, index,atr_multiple,look_back):   #price-time 
    body_sizes = (dataframe['close'] - dataframe['open']).loc[index - look_back:index].abs()
    average_body_size = body_sizes.mean()
    current_body_size = abs(dataframe['close'].loc[index] - dataframe['open'].loc[index])
    return current_body_size > atr_multiple * average_body_size

im_dic = {}

def imbalance_zone(imbalance_candle,df,look_back):
    verify_candle  = imbalance_candle+1
    for i in range(look_back):
        if is_body_size_greater_than_3x_average(df,imbalance_candle,3,look_back):
            upper = min(df["low"].loc[imbalance_candle-30:imbalance_candle-1])
            lower = df["high"][verify_candle] 
            if (upper-lower) > 0 :
                im_dic[ df.loc[imbalance_candle]["date"]] = [upper , lower]
            else :
                return
        else:
            return
        
retracement_after_n_candle = []

def retrace(df, imbalance_time , zone): 
    if True:
        ret_count = 0
        for i in imbalance_time:
            zone_low = zone[i][1]
            zone_high = zone[i][0]
            half_zone = (zone_low + zone_high)/2
            imb_price = df[df["date"]== i].close
            imb_index = df[df["date"]== i].index[0]

            if True: #imb_index+1 < len(df):
                if float(df.loc[imb_index+1].close) > float(df.loc[imb_index].close):
                    for index, row in df.loc[imb_index+1:].iterrows():
                        if half_zone >= float(row.close):
                            ret_count += 1
                            retracement_after_n_candle.append(index - imb_index)
                            im_dic[i].append(index)
                            break
                else:
                    for index, row in df.loc[imb_index+1:].iterrows():
                        if half_zone <= float(row.close):
                            ret_count += 1
                            retracement_after_n_candle.append(index - imb_index)
                            im_dic[i].append(index)
                            break
        print('Number of retracement: ', ret_count , 'out of :', len(imbalance_time))

        
for i in imb_candle_time:
#     print(i)
#     print(df)
#     print(df[df["date"]== i])
    ind = df[df["date"]== i].index[0]
    imbalance_zone(ind, df , 30)
    
def filter_non_balance_candle(imb_candle_time): 
    imx = []
    for i in imb_candle_time:
        if i in im_dic.keys():
            imx.append(i)
        
    return imx

retrace(df , filter_non_balance_candle(imb_candle_time) ,  im_dic)

# print(retracement_after_n_candle)
print('Total imbalance candle given:' , len(imb_candle_time),  '& Condition Satisfies',len(im_dic))

t = 0
for i in im_dic:
    im_dic[i].append(int(retracement_after_n_candle[t]))
    im_dic[i].append(abs(df[df['date'] == i].open - df[df['date'] == i].close))
    t +=1

print('imc_dic ::: Key: imbalance candle , Values = [low_zone , high_zone , retracement_after_n_candle , Imbalance_candle size]')
print(im_dic) 
    
#Key: imbalance candle , Values = [low_zone , high_zone , retracement_after_n_candle , Imbalance_candle size]
rt = retracement_after_n_candle
rt.sort()
retrace_dic = {}
for i in rt:
    if i not in retrace_dic:
        retrace_dic[i] = 1
    
    else:
        retrace_dic[i] +=1 
        
print('--------------------------------------------')
print('Dictionary of retracement' , retrace_dic)


Number of retracement:  230 out of : 230
Total imbalance candle given: 453 & Condition Satisfies 230
imc_dic ::: Key: imbalance candle , Values = [low_zone , high_zone , retracement_after_n_candle , Imbalance_candle size]
{'2016-11-01 14:23:00': [1078.55, 1078.0, 137, 2, 135    2.3
dtype: float64], '2016-11-02 09:15:00': [1072.2, 1064.8, 195, 13, 182    4.05
dtype: float64], '2016-11-03 13:12:00': [1044.5, 1043.85, 387, 1, 386    2.4
dtype: float64], '2016-11-03 13:17:00': [1042.55, 1042.2, 390, 1, 389    2.25
dtype: float64], '2016-11-10 09:39:00': [1037.1, 1032.2, 1104, 39, 1065    6.55
dtype: float64], '2016-11-17 09:31:00': [930.1, 929.0, 1674, 4, 1670    8.05
dtype: float64], '2016-11-17 13:50:00': [920.0, 918.25, 1739, 1, 1738    3.45
dtype: float64], '2016-11-18 11:35:00': [915.5, 913.7, 1833, 1, 1832    4.2
dtype: float64], '2016-11-25 12:30:00': [945.8, 945.5, 2658, 1, 2657    2.6
dtype: float64], '2016-11-28 13:36:00': [939.45, 939.1, 2848, 1, 2847    2.85
dtype: float64], '2

In [32]:
def report(retracement_after_n_candle , retrace_dic):
    rt = np.array(retracement_after_n_candle)
    dd = []
    for i in retracement_after_n_candle:
        if i>5:
            dd.append(i)
    dd = np.array(dd)
    print('Total signal: ', len(im_dic))
    print('Total Retracement' , len(retracement_after_n_candle))
    print('---------------')
    print('1 candle retrace ', retrace_dic[1] , ':' , retrace_dic[1]*100/len(retracement_after_n_candle), '%')
    print('2 candle retrace ', retrace_dic[2], ':' , retrace_dic[2]*100/len(retracement_after_n_candle), '%')
    print('3 candle retrace ', retrace_dic[3], ':' , retrace_dic[3]*100/len(retracement_after_n_candle), '%')
    print('4 candle retrace ', retrace_dic[4], ':' , retrace_dic[4]*100/len(retracement_after_n_candle), '%')
    print('5 candle retrace ', retrace_dic[5], ':' , retrace_dic[5]*100/len(retracement_after_n_candle), '%')
    print('------------')
    print('>5 candle retrace ', (len(retracement_after_n_candle) - retrace_dic[1]-retrace_dic[2]-retrace_dic[3]-retrace_dic[4]-retrace_dic[5])*100/len(retracement_after_n_candle),'%')
    print('Mean of retracement: ', rt.mean())
    print('std of retracement: ', rt.std())
    print('95 Percentile', np.percentile(rt,95))
    print('90 Percentile', np.percentile(rt,90))
    print('75 Percentile', np.percentile(rt,75))
    print('---Excluding first 5---------------------')
    print('95 Percentile', np.percentile(dd,95))
    print('90 Percentile', np.percentile(dd,90))
    print('75 Percentile', np.percentile(dd,75))

report(retracement_after_n_candle, retrace_dic)

Total signal:  230
Total Retracement 230
---------------
1 candle retrace  93 : 40.43478260869565 %
2 candle retrace  13 : 5.6521739130434785 %
3 candle retrace  8 : 3.4782608695652173 %
4 candle retrace  9 : 3.9130434782608696 %
5 candle retrace  9 : 3.9130434782608696 %
------------
>5 candle retrace  42.608695652173914 %
Mean of retracement:  86.8913043478261
std of retracement:  381.04230212844107
95 Percentile 344.6499999999986
90 Percentile 68.89999999999995
75 Percentile 17.0
---Excluding first 5---------------------
95 Percentile 1303.1999999999996
90 Percentile 415.59999999999985
75 Percentile 57.25


In [33]:
def plot(df3,ds=None):
    
    # Create the candlestick chart
    fact = 1
    candlestick = go.Candlestick(x=df3.index, open=df3['open']*fact, close=df3['close']*fact, high=df3['high']*fact, low=df3['low']*fact)
#     line  = go.bar()
    # Add a line plot with secondary x-axis
    sd = []
    if (ds) != None:
        line = go.Scatter(x=ds[ds["tt"]].index, y=ds[ds["tt"]]['close'], name='signal', xaxis='x2')
        sd.append(line)
    #line = go.Bar(x=df3.index, y=(df3['volume']/df3["volume"].mean())*data["close"].mean()/3, name='Volume')
    sd.append(candlestick)
    # Create the figure with both the candlestick and line plots
    fig = go.Figure(data=sd)

    # Update the layout to add a secondary x-axis
#     fig.update_layout(xaxis2=dict(title='signal', overlaying='x', side='top'),yaxis2 = dict(title='volume', overlaying='y', side='top'))
    fig.update_layout(xaxis_rangeslider_visible=False)

    # Show the chart
    fig.show()
    return

In [34]:
i = 143795
check_df = df.iloc[i-4:i+10]
plot(check_df)

In [35]:
# im_dic['2016-12-28 13:18:00']

In [36]:
df[df['date'] == '2021-04-19 12:25:00']

,index,date,open,high,low,close,volume_per_candle,volume,timeDiff,start,end
143795,538742,2021-04-19 12:25:00,2580.0,2580.55,2573.8,2573.8,4210.2275,19594.0,0.0,1.618815e+09,1.618815e+09


In [20]:
df[i-3:i+10]

,index,date,open,high,low,close,volume_per_candle,volume,timeDiff,start,end
134058,514302,2021-01-01 15:04:00,2784.95,2785.00,2783.00,2784.55,3662.1850,5989.0,60.0,1.609494e+09,1.609494e+09
134059,514304,2021-01-01 15:06:00,2783.35,2784.80,2783.00,2784.00,3662.1850,6245.0,60.0,1.609494e+09,1.609494e+09
134060,514305,2021-01-01 15:07:00,2783.05,2784.00,2783.00,2783.05,3662.1850,8705.0,0.0,1.609494e+09,1.609494e+09
134061,514306,2021-01-01 15:08:00,2783.00,2783.05,2775.25,2776.55,3675.5675,37989.0,0.0,1.609494e+09,1.609494e+09
134062,514307,2021-01-01 15:09:00,2775.55,2777.95,2774.10,2777.45,3685.4100,12847.0,0.0,1.609494e+09,1.609494e+09
134063,514308,2021-01-01 15:10:00,2777.50,2778.80,2776.00,2776.00,3728.9275,10495.0,0.0,1.609494e+09,1.609494e+09
134064,514309,2021-01-01 15:11:00,2776.30,2779.75,2776.00,2778.05,3774.7550,9800.0,0.0,1.609494e+09,1.609494e+09
134065,514311,2021-01-01 15:13:00,2778.05,2779.00,2777.00,2777.90,4250.6600,6719.0,60.0,1.609494e+09,1.609494e+09
134066,514312,2021-01-01 15:14:00,2777.85,2778.50,2775.35,2776.75,4912.1075,7668.0,0.0,1.609494e+09,1.609494e+09
134067,514313,2021-01-01 15:15:00,2777.10,2777.35,2773.85,2775.55,5377.7225,10470.0,0.0,1.609494e+09,1.609494e+09


In [21]:
im_dic["2021-01-01 15:08:00"]

[2782.0,
 2777.95,
 134062,
 1,
 134061    6.45
 dtype: float64]

In [46]:
# im_dic

In [16]:
im_dic

{'2016-11-01 14:23:00': [1078.55,
  1078.0,
  137,
  2,
  135    2.3
  dtype: float64],
 '2016-11-02 09:15:00': [1072.2,
  1064.8,
  195,
  13,
  182    4.05
  dtype: float64],
 '2016-11-03 13:12:00': [1044.5,
  1043.85,
  387,
  1,
  386    2.4
  dtype: float64],
 '2016-11-03 13:17:00': [1042.55,
  1042.2,
  390,
  1,
  389    2.25
  dtype: float64],
 '2016-11-10 09:39:00': [1037.1,
  1032.2,
  1104,
  39,
  1065    6.55
  dtype: float64],
 '2016-11-17 09:31:00': [930.1,
  929.0,
  1674,
  4,
  1670    8.05
  dtype: float64],
 '2016-11-17 13:50:00': [920.0,
  918.25,
  1739,
  1,
  1738    3.45
  dtype: float64],
 '2016-11-18 11:35:00': [915.5,
  913.7,
  1833,
  1,
  1832    4.2
  dtype: float64],
 '2016-11-25 12:30:00': [945.8,
  945.5,
  2658,
  1,
  2657    2.6
  dtype: float64],
 '2016-11-28 13:36:00': [939.45,
  939.1,
  2848,
  1,
  2847    2.85
  dtype: float64],
 '2016-12-01 14:21:00': [940.0,
  939.6,
  3268,
  1,
  3267    2.0
  dtype: float64],
 '2016-12-02 09:15:00': [936